In [ ]:
import pandas as pd
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [ ]:
passing = 'NFL_STATISCS/Career_Stats_Passing.csv'
reciving = 'NFL_STATISCS/Career_Stats_Receiving.csv'
rushing = 'NFL_STATISCS/Career_Stats_Rushing.csv'
qb = 'NFL_STATISCS/Game_Logs_Quarterback.csv'
basic = 'NFL_STATISCS/Basic_Stats.csv'
runnungBack = 'NFL_STATISCS/Game_Logs_Runningback.csv'
receiver = 'NFL_STATISCS/Game_Logs_Wide_Receiver_and_Tight_End.csv'

In [ ]:
#passing_df = pd.read_csv(file_passing)
#passing_df.loc[passing_df['Player Id']=='billdemory/2512778',:]

In [ ]:
# Create all the DataFrames
qb_df = pd.read_csv(qb)             # Len = 40,247
runnungBack_df = pd.read_csv(runnungBack) #Len = 67,661   nan, 'RB', 'FB'    #11,626 That Contain Position 
receiver_df = pd.read_csv(receiver)       #Len 112,632    nan, 'WR', 'TE',   # 24,786 That Contain Position 


passing_df = pd.read_csv(passing) # Len = 8,525 First Year = 1924
rushing_df = pd.read_csv(rushing) # Len = 17,507 First Year = 1924
reciving_df = pd.read_csv(reciving) #Len = 18,128
basic_df = pd.read_csv(basic)       #Len = 17,172



In [ ]:
#Cleaning the QB Data Frame

qb_df['Position'] = qb_df['Position'].fillna('QB')

qb_clean_df = qb_df.loc[(qb_df['Season']=='Regular Season') | (qb_df['Season']=='Postseason'),:].replace('--',0)
qb_clean_df

qb_clean_df = qb_clean_df.rename(columns = {'Player Id': 'Player_id',
                                           'Games Played':'Games_Played',
                                          'Games Started':'Games_Started',
                                            'Passes Completed' :'Passes_Completed',
                                            'Passes Attempted':'Passes_Attempted'})


columns = ['Player_id', 'Name', 'Position','Year', 'Season', 'Games_Played', 
           'Games_Started', 'Passes_Completed', 'Passes_Attempted']
qb_clean_df = qb_clean_df.reset_index(drop= True)
qb_df_sql = qb_clean_df.loc[:,columns]

qb_df_sql

In [ ]:
#DATA FRAME THAT contains unique players

qb_id = qb_df_sql['Player_id'].unique()
qb_name = qb_df_sql['Name'].unique()

#qb_id_name = pd.DataFrame(qb_id,qb_name, columns  = ['Player_id','Player_id'])
qb_id_name = pd.DataFrame({'Player_id': qb_id, 'Name' : qb_name})#, columns  = ['Player_id','Player_id'])
qb_id_name


In [ ]:
# Cleaning RB Data Frame UNIQUE QB AND NANMES

rb_df = runnungBack_df.loc[(runnungBack_df['Position']== 'RB') | (runnungBack_df['Position']== 'FB'),:]
rb_df
runnungBack_clean_df = rb_df.loc[(rb_df['Season']=='Regular Season') | (rb_df['Season']=='Postseason'),:].reset_index(drop=True)
runnungBack_clean_df

In [ ]:
#receiver_df = pd.read_csv(receiver)       #Len 112,632    nan, 'WR', 'TE',   # 24,786 That Contain Position 


rec_df = receiver_df.loc[(receiver_df['Position']== 'WR') | (receiver_df['Position']== 'TE'),:]
rec_df
receiver_clean_df = rec_df.loc[(rec_df['Season']=='Regular Season') | (rec_df['Season']=='Postseason'),:].reset_index(drop=True)
receiver_clean_df


In [ ]:
conn_string = 'postgres:postgres@localhost:5432/NFL_Statistics'
engine = create_engine(f'postgresql://{conn_string}')
Base = declarative_base()

In [ ]:
class QB_NAMES(Base):
    __tablename__ = 'QB_NAMES'
    Player_id = Column(String,primary_key = True)
    Name = Column(String)

class RB(Base):
    __tablename__ = 'RB'
    Player_id = Column(String, primary_key = True)
    name = Column(String)
    Position = Column(String)
    Year = Column(Integer)
    Season = Column(String)
    Games_Played = Column(Integer)
    Games_Started = Column(Integer)
    Rushing_Yards = Column(Integer)
    Rushing_Attempts = Column(Integer)

In [ ]:
Base.metadata.tables

In [ ]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import Session
session = Session(engine)

In [ ]:
pd.read_sql('SELECT * FROM "QB_NAMES"', engine)


In [ ]:
session.new

In [ ]:
#QB_NAMES
qb_id_name.to_sql(name="QB_NAMES",con=engine,if_exists='replace', index=False)

In [ ]:
qb_id_name

In [ ]:

session.close()